# 강의

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import warnings

warnings.filterwarnings("ignore")

In [24]:
url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1")
html = BeautifulSoup(url.text)

In [25]:
def make_table(html):
    table  = html.find("table",{"class":"type_2"})
    table = pd.read_html(str(table))[0] #리스트 안에 있는 표 꺼내기
    del table['토론실']
    del table['N']
    table = table.dropna(subset = "종목명")
    return table

In [28]:
kospi_page = int(html.find("td", class_="pgRR").find("a")['href'].split("=")[-1])

kospi_page

46

In [30]:
from tqdm import tqdm

kospi_box = []
for n in tqdm(range(1,kospi_page+1)):
    url = requests.get(f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={n}")
    html = BeautifulSoup(url.text)
    table = make_table(html)
    kospi_box.append(table)
    time.sleep(2)

100%|██████████| 46/46 [02:19<00:00,  3.02s/it]


In [32]:
kospi = pd.concat(kospi_box, ignore_index = True)
kospi['소속'] = '코스피'
kospi.to_excel("kospi.xlsx")
kospi

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,소속
0,삼성전자,61500.0,"하락 2,700",-4.21%,100.0,3671416.0,5969783.0,53.94,32184989.0,15.03,4.15,코스피
1,SK하이닉스,174600.0,"하락 9,200",-5.01%,5000.0,1271092.0,728002.0,53.98,4517025.0,50.87,-15.61,코스피
2,LG에너지솔루션,414500.0,보합0,0.00%,500.0,969930.0,234000.0,4.97,443279.0,-577.30,6.36,코스피
3,삼성바이오로직스,977000.0,"하락 9,000",-0.91%,2500.0,695370.0,71174.0,13.32,128088.0,67.62,9.12,코스피
4,현대차,244000.0,"하락 10,500",-4.13%,5000.0,510976.0,209416.0,41.40,768774.0,5.29,13.68,코스피
...,...,...,...,...,...,...,...,...,...,...,...,...
2249,RISE 미국장기국채선물인버스(H),10135.0,하락 5,-0.05%,0.0,20.0,200.0,0.00,3.0,NaN,NaN,코스피
2250,RISE 차이나H선물인버스(H),9885.0,하락 465,-4.49%,0.0,20.0,200.0,0.00,13959.0,NaN,NaN,코스피
2251,TIGER BBIG레버리지,2255.0,상승 70,+3.20%,0.0,19.0,850.0,0.00,11314.0,NaN,NaN,코스피
2252,KOSEF 차이나A50커넥트인버스MSCI(합성 H),9200.0,하락 375,-3.92%,0.0,18.0,200.0,0.00,16.0,NaN,NaN,코스피


# 실습

In [34]:
#코스피
ur1 = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1")
#코스닥
url2 = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page=1")

html = BeautifulSoup(url.text)

In [35]:
def make_table(html):
    table  = html.find("table",{"class":"type_2"})
    table = pd.read_html(str(table))[0] #리스트 안에 있는 표 꺼내기
    del table['토론실']
    del table['N']
    table = table.dropna(subset = "종목명")
    return table

In [38]:
kospi_page = int(html.find("td", class_="pgRR").find("a")['href'].split("=")[-1])

kosdaq_page = int(html.find("td", class_="pgRR").find("a")['href'].split("=")[-1])

max_page = max(kospi_page, kosdaq_page)

In [40]:
from tqdm import tqdm

kospi_box = []
kosdaq_box = []

for n in tqdm(range(1, max_page + 1)):
    if n <= kospi_page:
        url1 = requests.get(f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={n}")
        html1 = BeautifulSoup(url1.text, "html.parser")
        table1 = make_table(html1)
        kospi_box.append(table1)

    if n <= kosdaq_page:
        url2 = requests.get(f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page={n}")
        html2 = BeautifulSoup(url2.text, "html.parser")
        table2 = make_table(html2)
        kosdaq_box.append(table2)

    time.sleep(2)

100%|██████████| 46/46 [03:05<00:00,  4.04s/it]


In [41]:
kospi_df = pd.concat(kospi_box, ignore_index=True)
kosdaq_df = pd.concat(kosdaq_box, ignore_index=True)

final_df = pd.concat([kospi_df, kosdaq_df], ignore_index=True)

In [42]:
final_df.to_excel('코스피&코스닥.xlsx', index=False)

In [43]:
final_df = pd.read_excel('코스피&코스닥.xlsx')

PER_df = final_df[(final_df['PER'] >= 1) & (final_df['PER'] <= 10)]

In [45]:
PER_df['PER'].describe()

,PER
count,666.000000
mean,6.052808
std,2.288311
min,1.000000
25%,4.235000
50%,5.995000
75%,7.987500
max,9.990000


In [46]:
PER_df.to_excel('PER.xlsx', index=False)

# 시세

파이썬 접근이 아니라 다른게 접근하는걸로 속이기

In [48]:
dic = {"User-Agent" : "mozilla/5.0 (macintosh; intel mac os x 10_15_7"}

url = requests.get("https://finance.naver.com/item/sise.naver?code=005930", headers = dic)
html = BeautifulSoup(url.text)

In [49]:
html

<!--  global include --><html lang="ko">
<head>
<title>삼성전자 : 네이버페이 증권</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="네이버페이 증권" name="apple-mobile-web-app-title"/>
<meta content="네이버페이 증권" property="og:title"/>
<meta content="https://ssl.pstatic.net/static/m/stock/im/2016/08/og_stock-200.png" property="og:image"/>
<meta content="https://finance.naver.com" property="og:url"/>
<meta content="국내 해외 증시 지수, 시장지표, 뉴스, 증권사 리서치 등 제공" property="og:description"/>
<meta content="article" property="og:type"/>
<meta content="" property="og:article:thumbnailUrl"/>
<meta content="네이버페이 증권" property="og:article:author"/>
<meta content="http://FINANCE.NAVER.COM" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20240926170906/css/finance_header.css" rel="stylesheet" type="text/css"/>
<link href=

In [ ]:
table  = html.find("table",{"class":"type2"})

# 배당금 수업

In [69]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import warnings

warnings.filterwarnings("ignore")

url = requests.get("https://finance.naver.com/sise/dividend_list.naver")
html = BeautifulSoup(url.text)

table = html.find("table", class_="type_1 tb_ty")

table = pd.read_html(str(table), header = 1)[0]

table

,종목명,현재가,기준월,배당금,수익률 (%),배당성향 (%),ROE (%),PER (배),PBR (배),1년전,2년전,3년전
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,한국패러랠,191.0,23.12,2168.0,1135.08,-,-,-,-,390,90,235
2,스타에스엠리츠,2780.0,23.12,961.0,34.57,220.22,5.24,26.77,1.51,1572,200,150
3,예스코홀딩스,45750.0,23.12,8750.0,19.13,157.91,4.99,8.38,0.30,2500,2250,2000
4,에이블씨엔씨,7750.0,24.04,1427.0,18.41,605.02,6.05,28.98,2.01,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
65,푸른저축은행,8880.0,23.12,650.0,7.32,48.35,5.05,8.22,0.32,650,650,550
66,세아특수강,13740.0,23.12,1000.0,7.28,-307.01,-0.82,-43.67,0.35,1000,1200,700
67,한화리츠,4185.0,24.04,304.0,7.26,345.90,-,112.24,1.07,126,-,-
68,NH투자증권우,11810.0,24.03,850.0,7.20,50.47,7.50,6.51,0.47,750,1100,750


# 배당금 실습

In [76]:
columns_to_replace = ['1년전','2년전','3년전']

table[columns_to_replace] = table[columns_to_replace].replace('-', 0)
table[columns_to_replace] = table[columns_to_replace].astype(int)

table

,종목명,현재가,기준월,배당금,수익률 (%),배당성향 (%),ROE (%),PER (배),PBR (배),1년전,2년전,3년전
2,스타에스엠리츠,2780.0,23.12,961.0,34.57,220.22,5.24,26.77,1.51,1572,200,150
3,예스코홀딩스,45750.0,23.12,8750.0,19.13,157.91,4.99,8.38,0.30,2500,2250,2000
4,에이블씨엔씨,7750.0,24.04,1427.0,18.41,605.02,6.05,28.98,2.01,0,0,0
8,신한알파리츠,6120.0,24.03,828.0,13.53,727.40,1.68,76.30,1.14,372,352,302
9,크레버스,16020.0,23.12,2000.0,12.48,110.97,33.62,12.71,3.15,1800,2000,800
10,정다운,2530.0,23.12,300.0,11.86,30.04,30.34,2.80,0.75,100,100,0
12,제이알글로벌리츠,3540.0,24.06,390.0,11.02,158.83,2.82,32.42,0.94,380,366,330
15,인화정공,20650.0,23.12,2250.0,10.90,817.65,1.47,43.53,0.63,0,0,0
16,마스턴프리미어리츠,2250.0,24.03,239.0,10.62,-20.96,-9.01,-7.18,0.72,263,0,0
17,하나투어,50500.0,24.04,5000.0,9.90,164.69,32.03,17.77,4.74,0,0,0


In [77]:
table = table[table['3년전'] <= table['2년전']].copy()
table.describe()

,현재가,기준월,배당금,수익률 (%),1년전,2년전,3년전
count,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000
mean,19688.409091,23.535000,1779.636364,9.563636,1097.681818,1048.795455,751.931818
std,47605.252969,0.463297,3918.383489,4.662960,2720.964029,2843.344817,2099.780192
min,2250.000000,23.060000,220.000000,7.200000,0.000000,0.000000,0.000000
25%,4341.250000,23.120000,374.500000,7.537500,242.500000,187.500000,0.000000
50%,8155.000000,23.120000,750.000000,8.010000,387.500000,383.000000,311.000000
75%,15547.500000,24.030000,1294.250000,9.067500,1012.500000,1025.000000,712.500000
max,316000.000000,24.070000,25000.000000,34.570000,18000.000000,19000.000000,14000.000000


In [86]:
table_df = table[table['2년전'] <= table['1년전']].copy()
table_df

,종목명,현재가,기준월,배당금,수익률 (%),배당성향 (%),ROE (%),PER (배),PBR (배),1년전,2년전,3년전,상승정도
2,스타에스엠리츠,2780.0,23.12,961.0,34.57,220.22,5.24,26.77,1.51,1572,200,150,1422
3,예스코홀딩스,45750.0,23.12,8750.0,19.13,157.91,4.99,8.38,0.30,2500,2250,2000,500
4,에이블씨엔씨,7750.0,24.04,1427.0,18.41,605.02,6.05,28.98,2.01,0,0,0,0
8,신한알파리츠,6120.0,24.03,828.0,13.53,727.40,1.68,76.30,1.14,372,352,302,70
10,정다운,2530.0,23.12,300.0,11.86,30.04,30.34,2.80,0.75,100,100,0,100
12,제이알글로벌리츠,3540.0,24.06,390.0,11.02,158.83,2.82,32.42,0.94,380,366,330,50
15,인화정공,20650.0,23.12,2250.0,10.90,817.65,1.47,43.53,0.63,0,0,0,0
16,마스턴프리미어리츠,2250.0,24.03,239.0,10.62,-20.96,-9.01,-7.18,0.72,263,0,0,263
17,하나투어,50500.0,24.04,5000.0,9.90,164.69,32.03,17.77,4.74,0,0,0,0
19,SK디앤디,8740.0,23.12,800.0,9.15,17.22,14.13,6.92,0.94,800,800,600,200


In [89]:
#3년연속 배당금이 상승한 종목 리스트
table_df['상승정도'] = table['1년전'] - table['3년전']

table_df[['종목명','상승정도']].sort_values(by = '상승정도', ascending = False)

,종목명,상승정도
23,일성아이에스,3850
54,광주신세계,1500
2,스타에스엠리츠,1422
30,넥스틸,603
3,예스코홀딩스,500
52,메가스터디,450
37,NH올원리츠,300
60,동국홀딩스,300
38,디앤디플랫폼리츠,300
24,미래에셋글로벌리츠,286
